In [ ]:
!pip install pdfplumber
!pip install PyPDF2
!pip install -U spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install -U spacy

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.3 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

In [ ]:
# -*- coding: utf-8 -*-
"""final.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1C1wUDLtt1oW1m5eDCMBDK4s_dr2sLJ2r
"""

# from google.colab import drive
# drive.mount('/content/drive')

# !unzip /content/drive/MyDrive/HireAIML.zip -d '/content/HireAIML'

#!unzip /content/drive/MyDrive/HIREAIML_sample.zip -d '/content/HIREAIML_sample'



"""Import dependencies
===================
"""

import math
import PyPDF2
import os
from os.path import isfile, join
from io import StringIO
import pandas as pd
from collections import Counter
import spacy
from spacy.matcher import PhraseMatcher
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import re
import pdfplumber

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_lg")

"""Extract Keywords from input_resumes
===================================
"""

# Function to read PDFs and extract text
def pdfextract(file):
    fileReader = PyPDF2.PdfReader(open(file, 'rb'))
    text = []
    for page in fileReader.pages:
        t = page.extract_text()
        text.append(t)
    return " ".join(text)

"""Extract Years of Experience from resumes
==========================================
"""

import re
from datetime import datetime

# Combined regex pattern for extracting years of experience
year_pattern = re.compile(
    r"(?P<years>\d+(?:\.\d+)?)(?:\+|-)?\s*(?:years?|yrs?\.?)?\s*(?:of\s*)?experience|"  # Match explicit years of experience with or without "years"
    r"(?P<start_month>[a-zA-Z]{3,9})?\s*(?P<start_year>\d{4})\s*[-to]+\s*(?P<end_month>[a-zA-Z]{3,9})?\s*(?P<end_year>\d{4})\s*(?:experience)?",  # Match date ranges followed optionally by "experience"
    re.IGNORECASE
)

# Function to calculate experience from text
def extract_years_of_experience(resume_text):
    total_years = 0.0

    # Match the combined pattern
    for match in year_pattern.finditer(resume_text):
        # Check if the match is for explicit years of experience
        if match.group("years"):
            years = float(match.group("years"))
            # print(f"Explicit Experience Match: {years} years")
            total_years += years

        # Check if the match is for a date range
        elif match.group("start_year") and match.group("end_year"):
            start_year = int(match.group("start_year"))
            end_year = int(match.group("end_year"))
            start_month = match.group("start_month") if match.group("start_month") else "Jan"
            end_month = match.group("end_month") if match.group("end_month") else "Dec"

            try:
                # Convert month abbreviations to title case to match datetime format
                start_month = start_month.capitalize()[:3]
                end_month = end_month.capitalize()[:3]

                # Parse dates - handling cases where only years are mentioned
                start_date = datetime.strptime(f"{start_month} {start_year}", "%b %Y")
                end_date = datetime.strptime(f"{end_month} {end_year}", "%b %Y")

                # Calculate the difference in years
                difference_in_years = (end_date - start_date).days / 365.25
                # print(f"Date Range Match: {difference_in_years:.2f} years from {start_date} to {end_date}")
                total_years += difference_in_years
            except ValueError as e:
                print(f"Error parsing date: {e}")

    # print(f"Total Extracted Years of Experience: {total_years:.2f} years")
    return total_years

# Function to match experience to score
def match_experience(years_of_experience, experience_map):
    for (min_years, max_years), score in experience_map.items():
        if min_years <= years_of_experience < max_years:
            return score
    return 0  # Default score if no range matches

"""Scoring on the basis of experience
==================================
"""

# Function to score resumes based on extracted keywords and education
def score_resume(resume_text, keyword_dict, data_map):
    score = 0
    matched_keywords = set()  # To ensure each keyword is only counted once

    years_of_experience = extract_years_of_experience(resume_text)

    # Add experience score directly based on extracted years
    experience_score = match_experience(years_of_experience, data_map["Experience"])
    score += experience_score

    # Check for each category
    for category, keywords in keyword_dict.items():
        for keyword in keywords:
            if keyword.lower() in resume_text.lower() and keyword.lower() not in matched_keywords:
                matched_keywords.add(keyword.lower())
                if category in data_map and category != "Experience":  # Skip experience as it's already added
                    if isinstance(data_map[category], dict):
                        for key in data_map[category]:
                            if key.lower() == keyword.lower():
                                score += data_map[category][key]
                    else:
                        score += data_map[category]

    return score

"""Keyword Matching
================
"""

import os

# Function that does phrase matching and builds a candidate profile
def create_profile(file, keywords_dict, data_map):
    resume_text = pdfextract(file)
    filename = os.path.basename(file)
    score = score_resume(resume_text, keywords_dict, data_map)
    return filename, score

# Function to create a scoring dictionary
def data_dict():
    global data_map

    data_map = {
        "Machine Learning": 5,
        "Deep Learning": 5,
        "NLP": 4,
        "Computer Vision": 3,
        "Programming Languages": 2,
        "Cloud & Deployment": 2,
        "Education Qualification": {
            "PhD": 5,
            "MTech": 4,
            "B.Tech": 3,
            "MCA": 4,
            "BCA": 2,
            "BSC":2,
            "B.CA":2
        },
        "Experience": {
            (0, 2.5): 2,
            (2.5, 5.5): 3,
            (5.5, float('inf')): 5
        },
        "Certification": 2
    }

# Load job description and extract keywords
def extract_job_description_keywords():
    keyword_dict = {
        "Machine Learning": ["machine learning", "clustering", "logistic regression", "classification", "scikit learn",
                             "data science methodology", "model selection", "model evaluation", "pyspark", "hadoop",
                             "big data pipelines"],
        "Data Science":["Junior Data Engineer","Data Engineer","Data Analysis", "Data Analytics", "Linear Regression",
                        "Logistic Regression"," Predictive Modeling", "Business Analytics" ],
        "Deep Learning": ["deep learning", "tensorflow", "keras", "pytorch", "neural networks", "cnn", "rnn", "gans",
                          "model deployment"],
        "NLP": ["natural language processing", "nlp", "nltk", "spacy", "beautiful soup", "transformers",
                "nlp architectures", "deploying nlp models"],
        "Computer Vision": ["computer vision", "object detection", "ocr", "opencv", "cnn"],
        "Programming Languages": ["python", "numpy", "pandas", "matplotlib", "seaborn", "sql", "mongodb", "nosql",
                                   "graphql"],
        "Cloud & Deployment": ["aws", "azure", "google cloud platform", "firebase", "django"],
        "Education Qualification": ["phd", "mtech", "mca", "bca", "b.tech", "bsc", "B.CA"],
    }

    return keyword_dict

"""Sorting and Scoring
===================
"""

# Tuple Sorting Function
def Sort_Tuple(tup):
    return sorted(tup, key=lambda x: x[1], reverse=True)

# Display all candidates and top 5 selected candidates
def Sel_printer(scores):
    # Print scores of all candidates
    print("\nScores of All Candidates:")
    all_scores_table = PrettyTable(['Filename', 'Score'])
    for filename, sc in scores:
        all_scores_table.add_row([filename, sc])
    print(all_scores_table)

    # Select and print the top 5 candidates
    selected = Sort_Tuple(scores)[:5]  # Select the top 5 candidates
    print("\nTop 5 Candidates:")
    top_scores_table = PrettyTable(['Filename', 'Score'])
    for filename, sc in selected:
        top_scores_table.add_row([filename, sc])
    print(top_scores_table)

"""Main Function
=============
"""

# Main function
def main():
    global data_map

    # Specify the path to the resumes folder
    resumes_folder = input("Enter the path to your resumes folder: ")

    # Validate the directory
    if not os.path.isdir(resumes_folder):
        print(f"Error: The path '{resumes_folder}' is not a valid directory.")
        return

    # Get all resume files in the specified directory
    onlyfiles = [os.path.join(resumes_folder, f) for f in os.listdir(resumes_folder) if os.path.isfile(os.path.join(resumes_folder, f))]

    # Extract job description keywords
    keywords_dict = extract_job_description_keywords()
    data_dict()

    scores = []

    for file in onlyfiles:
        filename, score = create_profile(file, keywords_dict, data_map)
        scores.append((filename, score))

    Sel_printer(scores)

if __name__ == '__main__':
    main()

Enter the path to your resumes folder: /usr/resumes/datasets/trainResumes

Scores of All Candidates:
+-------------------+-------+
|      Filename     | Score |
+-------------------+-------+
| candidate_124.pdf |   24  |
| candidate_049.pdf |   11  |
| candidate_012.pdf |   14  |
| candidate_060.pdf |   32  |
| candidate_129.pdf |   26  |
| candidate_079.pdf |   12  |
| candidate_010.pdf |   19  |
| candidate_144.pdf |   11  |
| candidate_105.pdf |   21  |
| candidate_101.pdf |   22  |
| candidate_149.pdf |   39  |
| candidate_046.pdf |   38  |
| candidate_086.pdf |   16  |
| candidate_136.pdf |   20  |
| candidate_065.pdf |   14  |
| candidate_116.pdf |   13  |
| candidate_025.pdf |   48  |
| candidate_001.pdf |   36  |
| candidate_076.pdf |   12  |
| candidate_093.pdf |   22  |
| candidate_023.pdf |   16  |
| candidate_013.pdf |   33  |
| candidate_043.pdf |   23  |
| candidate_097.pdf |   9   |
| candidate_110.pdf |   18  |
| candidate_083.pdf |   19  |
| candidate_007.pdf |   14  |